In [1]:
## cremao_permit

# module
import psycopg2
import pandas as pd
import pymysql
import numpy as np
from shapely.wkb import loads

In [2]:
## 1.Setting

# 기준월
date = '202211'

# 광역시도코드
lawd_cd = ['11','26','27','28','29','30','31','36','41','42','43','44','45','46','47','48','50']

# Table 정보
dw_table = 'cremao_permit'
# ncp_table = 'permit'
columns = ['base_ym','pnu','juso','jimok','area','prpos_area_dstrc_nm','poly','use_yn','owner_gb','own_change_date','owner_sum','land_fig_nm','land_height_nm','road_fig_nm','land_use_nm']

In [ ]:
## 2. M2 TABLE 생성

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# DELETE
aws_cursor.execute("delete from m2." + dw_table + " where 1=1;")
aws_conn.commit()

# M2.CREMAO_PERMIT INSERT
for i in lawd_cd:
    aws_cursor.execute(
    """
    insert into m2.""" + dw_table + """ (
    
    
    
    """
    )
    aws_conn.commit()
    print(i, 'Done')

In [3]:
## 3. NCP 이관

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup) for tup in tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# insert
for i in lawd_cd:
    sql = "select * from m2." + dw_table + " where base_ym = '" + str(date) + "' and substring(pnu, 1, 2) = '" + str(i) + "' order by pnu;"
    aws_cursor.execute(sql)
    aws_conn.commit()
    rows = aws_cursor.fetchall()
    
    # dataframe
    df = pd.DataFrame(rows1, columns = columns)
    
    # null
    df = df.replace({np.nan: None})
    
    # val
    val = '(' + ('%s,' * len(df.columns))[:-1] + ')'
    
    # NCP insert
    for j in range(0, len(df), 1000):
    
        # DB Connect
        while True:
            try:
                conn = pymysql.connect(
                    host='db-bo6fn.pub-cdb.ntruss.com',
                    user='cremao',
                    password='cremao1234!!',
                    db='cremao',
                    charset='utf8'
                )
                break
            except Exception as err:
                pass

        # 1000개 단위로 나누기
        tmp = df[j:j + 1000]

        # insert
        execute_mogrify(conn, tmp, ncp_table, val)
        tmp = pd.DataFrame()

    # row count
    print(i, len(df))

11 911991
26 719310
27 602659
28 671673
29 391203
30 292897
31 504159
36 203320
41 5118995
42 2698190
43 2371001
44 3711711
45 3854192
46 5860288
47 5842377
48 4811532
50 873836
